# Setep 1 - Scraping

In [1]:
#Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import re
from webdriver_manager.chrome import ChromeDriverManager
import time

## NASA Mars News
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [8]:
# Set Executable Path & Initialize Chrome Browser
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [D:\Users\rodrigo.guarneros.IFT\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [9]:
# defining url and objects to scrap
url = "https://mars.nasa.gov/news/"
# variables: news_title and news paragraph
# making a request
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

#### NASA Mars News

In [12]:
# latest News Title 
#Alternative
#news_titles = []
#for x in soup.find_all("div", class_="content_title"):
#    news_titles.append(x.text)
#news_titles[1]
news_paragraph = ""
    # Repeating process for the paragraph string because the first time it fails in load the complete code
while news_paragraph == "":
    browser.visit("https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest")
    browser.is_element_present_by_tag('LI', wait_time=10)
    html = browser.html
    soup = bs(html, "html.parser")
# Storing First image, header and brief from the first News element.
# We get the second [1] element for the header because the first one with the same class is not related to any News
    try:
        news_header = soup.find_all("div",class_="content_title")[1].text.strip()
        news_paragraph =  soup.find("div",class_="article_teaser_body").text.strip()
    except:
        news_paragraph = ""
print(news_header)
print(news_paragraph)

5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover
The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.


#### JPL Mars Space Images - Featured Image

In [15]:
# Visit the url for JPL Featured Space Image here.
# defining url and objects to scrap
url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url2)
html2 = browser.html
soup2 = bs(html2, "html.parser")

In [26]:
#Use splinter to navigate the site and find the image 
# url for the current Featured Mars Image and assign 
# the url string to a variable called featured_image_url.
jpl_mainpage = "https://www.jpl.nasa.gov"
# We get the a tag with a unique id, and then we extract the first element containing the href link
featured_image = soup2.find_all("a", id="full_image")
featured_image = jpl_mainpage + featured_image[0]["data-fancybox-href"]
main_dict["ft_img"] = featured_image
featured_image

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19048_ip.jpg'

#### Mars Weather

In [18]:
# visiting the URL
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
# Parse Results HTML with BeautifulSoup
html3 = browser.html
weather = bs(html3, "html.parser")

In [20]:
# Looking for the "Insight sol" string 
latest_tweet = "None"
browser.visit("https://twitter.com/marswxreport?lang=en")
browser.execute_script("window.scrollTo(1, document.body.scrollHeight);")
# Checking if latest_tweet already has the content need
while latest_tweet == "None":
    time.sleep(2)
    # Scrolling down 
    browser.execute_script("window.scrollTo(1, document.body.scrollHeight);")
    html = browser.html
    soup = bs(html, "html.parser")
    # The list to iterate from all the tags
    article_list = []
# find all the tags with this specifications
    articles = soup.find_all("span", class_="css-901oao")

# getting text of each class with that name
    for x in articles:
        article_list.append(x.text)

    for y in article_list:
        main_dict ={}
        if y[:11] == "InSight sol":
            latest_tweet = y
            main_dict["last_tweet"] = latest_tweet
            print(latest_tweet)
            break
        else:
            latest_tweet = "None"

InSight sol 699 (2020-11-13) low -97.8ºC (-144.1ºF) high -9.9ºC (14.1ºF)
winds from the WNW at 4.2 m/s (9.4 mph) gusting to 14.2 m/s (31.8 mph)
pressure at 7.30 hPa


#### Mars Facts

In [20]:
# Visit the Mars Facts webpage here and use Pandas to scrape the table 
# containing facts about the planet including Diameter, Mass, etc.
url4 = "https://space-facts.com/mars/"
facts = pd.read_html(url4)[0]
facts.columns=["Variable", "Value"]
facts.set_index("Variable")

,Value
Variable,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


#### Mars Hemispheres

In [21]:
# obtain high resolution images for each of Mar's hemispheres.
# ou will need to click each of the links to the hemispheres 
# in order to find the image url to the full resolution image
url5 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url5)
html5 = browser.html
soup5 = bs(html5, "html.parser")
results = soup5.find_all("div",class_="item")

In [24]:
# Dictionary with the image url string and the hemisphere title to a list

hemisphere_image_urls = []

for img in results:
    hemispher={}
    title=img.find("div", class_="description")
    img_url=img.a["href"]
    print("--------------")
    print(title.a.text)
    print(f"https://astrogeology.usgs.gov/{img_url}")
    hemispher["title"]=title.a.text
    hemispher["img_url"]=f"https://astrogeology.usgs.gov{img_url}"
    hemisphere_image_urls.append(hemispher)

--------------
Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced
--------------
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced
--------------
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced
--------------
Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced
